In [2]:

import unittest

import numpy as np

from numpy.testing import assert_array_almost_equal

from skgstat.estimator import matheron, cressie, dowd, genton, minmax, percentile, entropy

In [4]:
import importlib.util
import sys

# For illustrative purposes.
package_name = 'scikit-gstat'

spec = importlib.util.find_spec(package_name)
if spec is None:
    print(package_name +" is not installed")

scikit-gstat is not installed


In [15]:
importlib.import_module(name, package=numbpy)

NameError: name 'name' is not defined

In [6]:
import skgstat

ModuleNotFoundError: No module named 'skgstat'

In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [4]:
import skfuzzy